In [8]:
import common
from common import *
from importlib import reload
reload(common)

C:\Users\FrankMohammed\Informatiekunde\InformatieVisualisatie\datavisualisatie\common.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



<module 'common' from 'C:\\Users\\FrankMohammed\\Informatiekunde\\InformatieVisualisatie\\datavisualisatie\\common.py'>

In [9]:
groupdivided_barchart()

In [10]:
boxplot_intstudents()

In [11]:
scatter_intscore_intstudent()

In [12]:
barchart_country_intstudents()

In [13]:
worldmap_intstudent('total_score', 'Gemiddelde totale score per land', 'Totale score', 100, False)

In [14]:
dotplot_totalscore()